<a href="https://colab.research.google.com/github/silverstar0727/pose-estimation/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.19.3
!pip install opencv-python==4.5.1.48
!pip install tqdm==4.56.0

!pip install mediapipe==0.8.3

In [42]:
import csv
import cv2
import numpy as np
import os
import sys
import tqdm
import pandas as pd

from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions import pose as mp_pose

from sklearn.neighbors import KNeighborsClassifier

In [43]:
def finetuning(csv_file="study_result.csv"):
    data = pd.read_csv(csv_file, header=None)
    x_train, y_train = data.iloc[:, 2:], data.iloc[:, 1]

    classifier = KNeighborsClassifier(n_neighbors = 3)

    classifier.fit(x_train, y_train)
    return classifier

classifier = finetuning()

In [64]:
def landmarks(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Initialize fresh pose tracker and run it.
    with mp_pose.Pose(upper_body_only=False) as pose_tracker:
        result = pose_tracker.process(image=frame)
        pose_landmarks = result.pose_landmarks
    
    # Save landmarks.
    if pose_landmarks is not None:
        # Check the number of landmarks and take pose landmarks.
        assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
        pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

        # Map pose landmarks from [0, 1] range to absolute coordinates to get
        # correct aspect ratio.
        frame_height, frame_width = frame.shape[:2]
        pose_landmarks *= np.array([frame_width, frame_height, frame_width])

        # Write pose sample to CSV.
        pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(np.float64).tolist()

        return pose_landmarks

In [77]:
input_frame = cv2.imread("image_154.jpg")
result = landmarks(input_frame)
result = np.array(result).reshape(1, -1)
print(classifier.predict(sample))

['do_not']


In [78]:
result.shape

(1, 99)